![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## Phidget_5 Graphs and gauges.

Using what we have learned so far, let's add some graphs to the display. This way we can see record over time of the temperature and humidity readings. 


## Libraries

We include a few libraries, for plotting and doing numerical work.

In [1]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

## Data storage

The data has to go somewhere, so we create a few buffers to store the temperature and humidity readings.

In [2]:
buffer_len = 100

h_buffer = np.zeros(buffer_len)
t_buffer = np.zeros(buffer_len)

## Gauges and charts

We create the two gauges for temperature and humidity, and the two charts to display the data over time. The gauges are called **t_indicator** and **h_indicator**, and the charts are called **t_chart** and **h_chart**.

We use the features of Plotly to put this four items into a single figure, using the **subplot()** function. We also add titles and some default ranges for the y-axes in the charts, to make this all work with the callback functions above. 

We then turn the figure into a live widget, using the **FigureWidget()** function.

Once these gauges and charts are all set up, we will connect them to the Phidget sensors.

In [3]:
x = np.arange(buffer_len)

t_indicator = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [5, 35]}}
)

h_indicator = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)


t_chart = go.Scatter(x=x, y=t_buffer, mode='lines')
h_chart = go.Scatter(x=x, y=h_buffer, mode='lines')


fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}],
           [{'type' : 'xy'}, {'type' : 'xy'}]])
fig.append_trace(t_indicator, row=1, col=1)
fig.append_trace(h_indicator, row=1, col=2)
fig.append_trace(t_chart, row=2, col=1)
fig.append_trace(h_chart, row=2, col=2)

fig.update_traces(name='Temperature', showlegend = True, row=2, col=1)
fig.update_traces(name='Humidity', showlegend = True, row=2, col=2)

fig.update_yaxes(title_text="Degrees C", range=[10, 30], row=2, col=1)
fig.update_yaxes(title_text="Percentage", range=[50, 70], row=2, col=2)

f = go.FigureWidget(fig)

f

FigureWidget({
    'data': [{'domain': {'x': [0.0, 0.45], 'y': [0.575, 1.0]},
              'gauge': {'axis': …

## Utility functions

We introduce a few utility functions, which are used to set the ranges on the vertical axes of our graphs. We don't know what temperature or humidity to expect, so we set the Y range to about $\pm 2$ degrees of the current temperature value, and $\pm 10$ percentage points for the humidity. 

This will make it easier for us to see small changes in the reading, reflected in the graphs.

In [4]:
## Utility function to bracket a value by plus/minus 2, 5 or 10

def bracket2(value):
    return (2*np.round(value/2)-2, 2*np.round(value/2)+2)

def bracket5(value):
    return (5*np.round(value/5)-5, 5*np.round(value/5)+5)

def bracket10(value):
    return (10*np.round(value/10)-10, 10*np.round(value/10)+10)

## Callback functions

Here we define the callback functions. These functions get called whenevern the Phidgets report new values for temperature and humidity. They are initiatied from the Javascript code above.

We have two buffers, or arrays, to store the temperature and humidity readings over a period of time. We chose 100 points, for simplicity, but you could change this. 

The callback functions do three things:
- update the value in the temperature or humidity gauge
- store the new value at the end of buffer, and display the 100 values in the temperature or humidity graph
- update the range of the y-axis for the graph, so it brackets the actual reading value.

As mentioned above, we do this bracketing so the graphs look nice, and we can see small changes in tmeperature or humidity.

Feel free to experiment on better ways to display the charts. 

In [5]:
def callback_temp(value):
    f.data[0]['value'] = value      
    t_buffer[0:-1] = t_buffer[1:]
    t_buffer[-1] = value
    f.data[2]['y'] = t_buffer
    f.layout['yaxis']['range'] = bracket2(value)

def callback_hum(value):
    f.data[1]['value'] = value
    h_buffer[0:-1] = h_buffer[1:]
    h_buffer[-1] = value
    f.data[3]['y'] = h_buffer
    f.layout['yaxis2']['range'] = bracket10(value)
 

## Setup the Phidget device

You must be running on Chrome on your laptop or desktop, and the Phidget device needs to be connected to the USB port on your computer. 

As in the earlier notebooks, the main steps are thus:

0. Load in some Python libraries
1. Load in the Phidgets library, in Javascript
2. Define and open the USB connection
3. Open the Humidity and Temperature device
4. Read values from the devices
5. When we are done, close the device


The Javascript code here is much like before, except at Step 3  we have introduced callback functions "callback_hum()" and "callback_hum()." A callback function is just a function that gets called whenever these JS function operate.

What is nice here is the callback functions are written in Python, so it is easy for us to program them in Python, and make changes as we need to. 

Step 4 then becomes "live" automatically. 

So run the following Python and Javascript code, and connect your Phidget. 

## Step 0.
Let's import a few Python libraries that we need. 

In [6]:
import ipywidgets as widgets
from IPython.display import Javascript, display
from time import sleep

## Step 1.
Now import the Phidgets library.

In [7]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});
require(['phidget22'], (phidget22) => {
   window.phidget22 = phidget22; 
});

<IPython.core.display.Javascript object>

In [8]:
## We pause for a second here, to allow the library to load
sleep(1)

## Step 2. 
We open the USB connection.

In [9]:
%%js

if (window.usbconn === undefined) {
    
    element.text("Creating a new USB Connection.");
    
    window.usbconn = new phidget22.USBConnection();
    
    usbconn.connect().then(() => {
        usbconn.requestWebUSBDeviceAccess();
    }).catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
}

<IPython.core.display.Javascript object>

In [10]:
## We rest for a bit while the USB connects
sleep(1)

### Confirm the connection 

In [11]:
%%js
element.text(usbconn.connected);

<IPython.core.display.Javascript object>

## Step 3.

Notice we introduce the callback functions here, at line 14 and 23. This is new, and a good trick to know aboutl 

In [12]:
%%js
// set up the humidity sensor
window.humSensor = new phidget22.HumiditySensor();
humSensor.onHumidityChange = function (humidity) {
    IPython.notebook.kernel.execute(
        "callback_hum(" + humidity + ");"
    );  
};
humSensor.open();
    
// set up the temperature sensor
window.tempSensor = new phidget22.TemperatureSensor();
tempSensor.onTemperatureChange = function (temperature) {
    IPython.notebook.kernel.execute(
        "callback_temp(" + temperature + ");"
    );  
};
tempSensor.open();

<IPython.core.display.Javascript object>

In [13]:
## We rest for a bit while the devices connect.
sleep(1)

## Step 4. PAUSE HERE

The gauges and graphs should be running now. Everything is live.  Scroll up and check it out the gauges, to see the temperature and humitity values. Try breathing on the sensor, so you can see changes in the readings. You should see both the gauges and the graphs change in real time. 

In [14]:
f

FigureWidget({
    'data': [{'domain': {'x': [0.0, 0.45], 'y': [0.575, 1.0]},
              'gauge': {'axis': …

## Step 5. Closing down the sensors

It is **really important** to close the sensor now, as otherwise they will keep busy forever, always trying to update the sliders with the latest values. So, don't skip this next step.

The following cell creates a button that you can click to close the Phidgets. Click it once you are all done with the Phidgets in this notebook. 

In [15]:
def doDisconnect(b):
    display(Javascript("""
        (async () => {
            await humSensor.close();
            await tempSensor.close();
            usbconn.close();
            usbconn.delete();
            delete window.usbconn;
            element.text("You have disconnected the Phidgets.");
        })();
    """))

run_button = widgets.Button(
    description = 'Click to disconnect'
)
print("Press this button when you are done, to disconnect the Phidgets")
run_button.on_click(doDisconnect)

display(run_button)

Press this button when you are done, to disconnect the Phidgets


Button(description='Click to disconnect', style=ButtonStyle())

<IPython.core.display.Javascript object>

### Confirm

You can confirm the phidget has been closed by running the following cell. 

If it is still attached (true), try clicking the button above, again.

In [17]:
%%js
element.text("Is the humidity sensor attached? " + humSensor.attached);
element.append("<p>Is the temperature sensor attached? " + tempSensor.attached);

<IPython.core.display.Javascript object>

## Going further

Some ideas to try on your own:

- is there a better way to bracket the values when we set the y-axis limits? Can we make the display nicer?
- can we add some "memory" to the y-axis ranges, so they don't jump around too much when the temperature or humidity reading crosses some critical points? For instance, when the temperature crosses an odd integer (e.g. value = 27), the range will jump around from (24,28) t0 (26,30). Which is a rather ugly display
- can we update the charts only once a second? That way it is easier to see the slow changes in value.
- can we record the x-axis as a time value, so we know at what time certain readings occur?
- can we save the data into a file, on some regular basis? This should include data and time stamps, so we have an accurate historical record of the reading.


## Conclusion

We have shown how to display the temperature and humidity values from the Phidget device using a live widget from the ipywidgets library. 

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)